# 🥋 Lekcja 32: Gradient Clipping (Ratunek przed wybuchem)

W głębokich sieciach (szczególnie RNN i Transformerach) zdarza się zjawisko **Exploding Gradients**.
Pochodna w jednym kroku wynosi np. `1000`. Wagi zmieniają się drastycznie. Sieć "wylatuje z toru" i zwraca `NaN`.

**Rozwiązanie: Gradient Clipping.**
Sprawdzamy **normę** (długość) wektora wszystkich gradientów.
Jeśli jest większa niż próg (np. 1.0), skalujemy wszystkie gradienty w dół, zachowując ich kierunek.

Wzór:
$$ g_{new} = g \cdot \frac{\text{max\_norm}}{\max(\text{max\_norm}, ||g||)} $$

PyTorch robi to jedną funkcją: `torch.nn.utils.clip_grad_norm_`.

In [1]:
import torch
import torch.nn as nn

# 1. Symulacja problemu (Wybuchający gradient)
# Prosta waga, która ma duży gradient
w = torch.tensor([10.0], requires_grad=True)

# Symulujemy stratę, która jest bardzo stroma
# loss = w^4 -> grad = 4*w^3
# dla w=10 -> grad = 4000
loss = w**4
loss.backward()

print(f"Gradient przed cięciem: {w.grad.item()}")

Gradient przed cięciem: 4000.0


## Clipping w akcji

Użyjemy `clip_grad_norm_`.
Ta funkcja działa **In-Place** na parametrach (modyfikuje `.grad` bezpośrednio).

In [2]:
# 2. Przycinanie
# max_norm=1.0 -> Chcemy, żeby długość wektora gradientów nie przekraczała 1.0
torch.nn.utils.clip_grad_norm_([w], max_norm=1.0)

print(f"Gradient po cięciu: {w.grad.item()}")

# Sprawdźmy, czy kierunek się zachował (dla skalara to tylko znak)
# Było 4000 (+), jest 1.0 (+). Jest ok.

Gradient po cięciu: 1.0


## Clipping w pętli treningowej (Wzorzec)

Gdzie wstawić clipping w kodzie?
**Pomiędzy** `backward()` a `step()`.

1.  `loss.backward()` (Policz gradienty).
2.  `clip_grad_norm_()` (Przytnij je, jeśli są za duże).
3.  `optimizer.step()` (Zrób krok z bezpiecznymi gradientami).

In [3]:
# Symulacja pętli z siecią RNN (które często wybuchają)
model = nn.RNN(input_size=10, hidden_size=20, batch_first=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# Losowe dane
inputs = torch.randn(5, 10, 10) # [Batch, Seq, Feat]
target = torch.randn(5, 20)     # [Batch, Hidden]

print("--- Pętla z Clippingiem ---")

for step in range(3):
    optimizer.zero_grad()
    
    output, _ = model(inputs)
    # Bierzemy ostatni krok czasu
    loss = (output[:, -1, :] - target).pow(2).mean()
    
    # 1. Liczymy gradienty
    loss.backward()
    
    # Sprawdźmy normę przed cięciem (dla ciekawości)
    # Obliczamy normę wszystkich parametrów naraz
    total_norm = torch.norm(torch.stack([torch.norm(p.grad.detach(), 2) for p in model.parameters() if p.grad is not None]), 2)
    print(f"Krok {step}: Norma gradientu = {total_norm:.4f}")
    
    # 2. PRZYCINAMY (Bezpiecznik)
    # Zazwyczaj max_norm ustawia się na 1.0 lub 5.0
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    
    # 3. Aktualizacja
    optimizer.step()

print("Trening stabilny.")

--- Pętla z Clippingiem ---
Krok 0: Norma gradientu = 0.7192
Krok 1: Norma gradientu = 0.7104
Krok 2: Norma gradientu = 0.6965
Trening stabilny.


## Clipping by Value vs by Norm

Są dwie metody:
1.  **`clip_grad_norm_` (Zalecane):** Skaluje cały wektor gradientów. Zachowuje **kierunek** update'u.
2.  **`clip_grad_value_`:** Ucina każdą liczbę z osobna (np. min -1, max 1). Zmienia kierunek wektora!

Zazwyczaj używamy **Norm**, bo chcemy iść w dobrą stronę, tylko wolniej.

In [5]:
# Demonstracja zmiany kierunku przy Value Clipping
g = torch.tensor([10.0, 1.0]) # Wektor [10, 1]. Kierunek dominuje oś X.

# Kopia do testów
g_norm = g.clone()
g_val = g.clone()

# 1. Norm Clipping (max_norm=5)
# Skalujemy cały wektor, żeby jego długość (hipotenusa) wynosiła 5.
# Proporcje 10:1 zostaną zachowane (kierunek ten sam).
torch.nn.utils.clip_grad_norm_([g_norm], max_norm=5.0)

# 2. Value Clipping (max_value=5)
# Zamiast clip_grad_value_ (które wymaga parametru z .grad), 
# używamy .clamp_, co robi matematycznie to samo na surowym tensorze.
# Ucinamy każdą liczbę, która jest większa niż 5 lub mniejsza niż -5.
g_val.clamp_(-5.0, 5.0) 

print(f"Oryginał: {g.tolist()}")
print(f"Po Norm Clip:  {g_norm.tolist()} (Proporcja zachowana - to jest bezpieczne)")
print(f"Po Value Clip: {g_val.tolist()}  (Kierunek ZMIENIONY! - 10 spadło do 5, a 1 zostało 1)")

Oryginał: [10.0, 1.0]
Po Norm Clip:  [10.0, 1.0] (Proporcja zachowana - to jest bezpieczne)
Po Value Clip: [5.0, 1.0]  (Kierunek ZMIENIONY! - 10 spadło do 5, a 1 zostało 1)


## 🥋 Black Belt Summary

1.  **Zawsze używaj `clip_grad_norm_`** przy trenowaniu **RNN, LSTM, GRU i Transformerów** (np. GPT). Te sieci są głębokie w czasie i gradienty lubią się tam kumulować.
2.  **